In [34]:
traditionalInterFile="output"
customInterFile="output1"
objectFile="assembled.obj"
PC=0
Name="PG1"
OPTAB={"ADD":{"Opcode":"18"},
       "AND":{"Opcode":"40"},
       "COMP":{"Opcode":"28"},
       "DIV":{"Opcode":"24"},
       "J":{"Opcode":"3C"},
       "JEQ":{"Opcode":"30"},
       "JGT":{"Opcode":"34"},
       "JLT":{"Opcode":"38"},
       "JSUB":{"Opcode":"48"},
       "LDA":{"Opcode":"00"},
       "LDB":{"Opcode":"68"},
       "LDCH":{"Opcode":"50"},
       "LDL":{"Opcode":"08"},
       "LDX":{"Opcode":"04"},
       "LPS":{"Opcode":"D0"},
       "MUL":{"Opcode":"20"},
       "OR":{"Opcode":"44"},
       "RD":{"Opcode":"D8"},
       "RSUB":{"Opcode":"4C"},
       "SSK":{"Opcode":"EC"},
       "STA":{"Opcode":"0C"},
       "STB":{"Opcode":"78"},
       "STCH":{"Opcode":"54"},
       "STI":{"Opcode":"D4"},
       "STL":{"Opcode":"14"},
       "STSW":{"Opcode":"E8"},
       "STX":{"Opcode":"10"},
       "SUB":{"Opcode":"1C"},
       "TD":{"Opcode":"E0"},
       "TIX":{"Opcode":"B8"},
       "WD":{"Opcode":"DC"}}


In [12]:
OPTAB["LDA"]dfsdfhsdhjfghdjsg

{'Opcode': '00'}

In [50]:
SYMTAB={}
def RESW(line,PC):
    number=int(line[2])
    PC+=number*3
    return PC
def WORD(line,PC):
    PC+=3
    return PC
def BYTE(line,PC):
    pcOffset=0
    string=line[2]
    operation=string[0]
    string=string.strip("'").strip("X").strip("C").strip("'")
    print("Operation=",operation," Data=",string)
    if(operation=='X'):
        pcOffset=len(string)/2
        if(len(string)%2):
            pcOffset=(len(string)+1)/2
    elif(operation=='C'):
        pcOffset=len(string)
    return int(PC+pcOffset)
PSEUDOOPTAB={
        "RESW":{"Subroutine":RESW},
        "WORD":{"Subroutine":WORD},
        "BYTE":{"Subroutine":BYTE}
}
def optabLookUp(opcode):
    for i in OPTAB:
        if(i==opcode):
           return OPTAB[i]["Opcode"]
def pseudooptabLookUp(pseudoopcode):
    for i in PSEUDOOPTAB:
        if(i==pseudoopcode):
           return PSEUDOOPTAB[i]["Subroutine"]
def symLookUpAndAdd(symbol,PC):
    for i in SYMTAB:
        if(i==symbol):
            return SYMTAB[i]["Address"]
    SYMTAB[symbol]={"Address":PC}
def symLookUp(symbol):
    for i in SYMTAB:
        if(i==symbol):
            return SYMTAB[i]["Address"]
print(SYMTAB)
def verify():
    count=0
    for i in OPTAB:
        count1=0
        for j in OPTAB:
            if(count==count1):
                continue
            else:
                if(i==j):
                    print("Name conflict:",i,"\t",j)
                else:
                    if(OPTAB[i]["Opcode"] == OPTAB[j]["Opcode"]):
                        print("Opcode conflict:",i,"\t",j)
            count1+=1
        count+=1
     
# 1st PASS
def passOne():
    f=open("/home/abhiram/SIC/"+input("Enter file name:"))
    data=f.read()
    data=data.split("\n")
    f=open(traditionalInterFile,"w")
    f1=open(customInterFile,"w")
    Name=""
    PC=0
    for i in data:
        test=("]"+i).split()
        test[0]=test[0].replace("]","")
        print(test)
        if(len(test)==1):
            continue
        if('.' in test[0]):
            continue
        if(test[1]=="START" and len(test)>2):
            PC=int("0x"+test[2],0)
            if(test[0]!=""):
                Name=test[0]
        elif(test[1]=="END" and len(test)>=2):
            print("Program END")
            f.write(hex(PC))
            f1.write(hex(PC))
            f.write("\t")
            f1.write("\t")
            for z in range(1,len(test)):
                f.write(test[z])
                if(z==1):
                    f1.write(test[z])
                else:
                    f1.write(test[z])
                f.write("\t")
                f1.write("\t")
            f.write("\n")
            f1.write("\n")
        else:
            if(test[0]!=""):
                symLookUpAndAdd(test[0],PC)
            opcode=optabLookUp(test[1])
            if(opcode==None):
                function=pseudooptabLookUp(test[1])
                if(function != None):
                    f.write(hex(PC))
                    f1.write(hex(PC))
                    f.write("\t")
                    f1.write("\t")
                    for z in range(1,len(test)):
                        f.write(test[z])
                        if(z==1):
                            f1.write(test[z])
                        else:
                            f1.write(test[z])
                        f.write("\t")
                        f1.write("\t")
                    f.write("\n")
                    f1.write("\n")
                    PC=function(test,PC)
            else:
                f.write(hex(PC))
                f1.write(hex(PC))
                f.write("\t")
                f1.write("\t")
                for z in range(1,len(test)):
                    f.write(test[z])
                    if(z==1):
                        f1.write(opcode)
                    else:
                        f1.write(test[z])
                    f.write("\t")
                    f1.write("\t")
                f.write("\n")
                f1.write("\n")
                PC+=3
                print(opcode)
    f.close()
    f1.close()
    if(Name==""):
        Name="PG1"
    return PC,Name
# Write symtab [Just for debugging]
def dumpSymtab():
    f=open("/tmp/symtab","w")
    for i in SYMTAB:
        f.write(i)
        f.write("\t")
        f.write(hex(SYMTAB[i]["Address"]))
        f.write("\n")
    f.close()
def zeroFill(data,length=6):
    if(len(data)<length):
        data="0"*(length-len(data))+data
    return data
def closeTextRecord(textRecord,f=None):
    start=textRecord.pop(0)
    outputString="T^"+zeroFill(start.replace("0x",""))+"^"+zeroFill(hex(len(textRecord)*3).replace("0x",""),2)
    for i in textRecord:
        outputString+="^"+i
    if(f!=None):
        f.write(outputString+"\n")
    return outputString
def createTextRecord(start):
    return [start]
def insertIntoTextRecord(textRecord,data,PC,f):
    if(len(textRecord)<11):
        textRecord.append(data)
    else:
        closeTextRecord(textRecord,f)
        textRecord=createTextRecord(PC)
        textRecord.append(data)
    return textRecord
def passTwo():
    print("Start Pass 2")
    f=open(customInterFile,"r")
    data=f.read()
    f.close()
    f=open(objectFile,"w")
    data=data.split("\n")
    start=data[0].split()[0].replace("0x","")
    start=zeroFill(start)
    print(data[0].split())
    headRecord="H^"+Name+"^"+start+"^"+zeroFill(hex(PC).replace("0x",""))
    f.write(headRecord+"\n")
    textRecord=None
    for i in data:
        i=i.split()
#         if(textRecord==None):
#             textRecord=createTextRecord(i[0])
        if(len(i)>1):
            if(i[1]=="RESW"):
                if(textRecord != None):
                    closeTextRecord(textRecord,f)
                    textRecord=None
#             elif(i[1]=="WORD"): 
#                 outputString=i[0].strip("0x")
#                 if(len(outputString+hex(i[2]))<6):
#                     offset=6-len(outputString+hex(i[2]))
#                     offsetData="0"*offset
#                     outputString+=offsetData
#                 outputString+=hex(i[2])
#                 if(textRecord==None):
#                     textRecord=createTextRecord(i[0])
#                 insertIntoTextRecord(textRecord,outputString,i[0],f)
            elif(i[1]=="WORD"):
                try:
                    outputData=int(i[2].strip())
                except:
                    outputData=0
                if(len(hex(outputData).replace("0x",""))<6):
                    outputData = zeroFill(hex(outputData).replace("0x",""))
                if(textRecord==None):
                    textRecord=createTextRecord(i[0])
                insertIntoTextRecord(textRecord,outputData,i[0],f)
            elif(i[1]=="BYTE"):
                try:
                    outputData=int(i[2].strip())
                except:
                    outputData=0
                if(len(hex(outputData).replace("0x",""))<6):
                    outputData = zeroFill(hex(outputData).replace("0x",""))
                if(textRecord==None):
                    textRecord=createTextRecord(i[0])
                insertIntoTextRecord(textRecord,outputData,i[0],f)
            elif(i[1]=="END"):
                print("WroteEnd Record")
                if(textRecord != None):
                    closeTextRecord(textRecord,f)
                try:
                    start=hex(symLookUp(i[2])).replace("0x","")
                except:
                    pass
                outputString="E^"+zeroFill(start)
                f.write(outputString+"\n")
            else:
                operand="0000"
                if(len(i)==3):
                    if("," in i[2]):
                        print("Unimplemented X indexing")
                    else:
                        try:
                            operand=hex(symLookUp(i[2])).replace("0x","")
                        except:
                            print("Symbol not found")
                instruction=i[1]+zeroFill(operand,4)
                if(textRecord==None):
                    textRecord=createTextRecord(i[0])
                print(instruction,i[1])
                insertIntoTextRecord(textRecord,instruction,i[0],f)
    f.close()
    return data


PC,Name=passOne()
dumpSymtab()
print("PC=",hex(PC))
# Cleanup variables 
data=passTwo()


{}
Enter file name:record.asm
['COPY', 'START', '1000']
['FIRST', 'STL', 'RETADR']
14
['CLOOP', 'JSUB', 'RDREC']
48
['', 'LDA', 'LENGTH']
00
['', 'COMP', 'ZERO']
28
['', 'JEQ', 'ENDFIL']
30
['', 'JSUB', 'WRREC']
48
['', 'J', 'CLOOP']
3C
['ENDFIL', 'LDA', 'EOF']
00
['', 'STA', 'BUFFER']
0C
['', 'LDA', 'THREE']
00
['', 'STA', 'LENGTH']
0C
['', 'JSUB', 'WRREC']
48
['', 'LDL', 'RETADR']
08
['', 'RSUB']
4C
['EOF', 'BYTE', "C'EOF'"]
Operation= C  Data= EOF
['THREE', 'WORD', '3']
['ZERO', 'WORD', '0']
['RETADR', 'RESW', '1']
['LENGTH', 'RESW', '1']
['BUFFER', 'RESB', '4096']
['RDREC', 'LDX', 'ZERO']
04
['', 'LDA', 'ZERO']
00
['RLOOP', 'TD', 'INPUT']
E0
['', 'JEQ', 'RLOOP']
30
['', 'RD', 'INPUT']
D8
['', 'COMP', 'ZERO']
28
['', 'JEQ', 'EXIT']
30
['', 'STCH', 'BUFFER,X']
54
['', 'TIX', 'MAXLEN']
B8
['', 'JLT', 'RLOOP']
38
['EXIT', 'STX', 'LENGTH']
10
['', 'RSUB']
4C
['INPUT', 'BYTE', "X'F1'"]
Operation= X  Data= F1
['MAXLEN', 'WORD', '4096']
['WRREC', 'LDX', 'ZERO']
04
['WLOOP', 'TD', 'OUTPUT']

In [43]:
SYMTAB

{'FIRST': {'Address': 4096},
 'CLOOP': {'Address': 4099},
 'ENDFIL': {'Address': 4117},
 'EOF': {'Address': 4138},
 'THREE': {'Address': 4141},
 'ZERO': {'Address': 4144},
 'RETADR': {'Address': 4147},
 'LENGTH': {'Address': 4150},
 'BUFFER': {'Address': 4153},
 '.SUBROUTINE': {'Address': 4153},
 'RDREC': {'Address': 4153},
 'RLOOP': {'Address': 4159},
 'EXIT': {'Address': 4183},
 'INPUT': {'Address': 4189},
 'MAXLEN': {'Address': 4190},
 'WRREC': {'Address': 4193},
 'WLOOP': {'Address': 4196},
 'OUTPUT': {'Address': 4217}}